## Step 1: Download Source Data
      
#### Standard HydroBASINS from HydroSHEDS: 
    * Store src HydroSHEDS data in repo under 'data/HydroSHEDS'
    * Individual file names need to remain as downloaded
    * Structure under this folder does not matter
    * See more information and download directions here:
    
#### Landscape Variables (var files):
    * Store src landscape variable (var) files in repo under 'data/var'
    * Structure under this folder does not matter
    * See more information and download directions for var files used in this assessment here:
    
#### Source File Information:
    * User created csv file containing processing information about each var file
    * For ease of use name and store file in the repo as: 'data/var/file_processing_info.csv'
    * Required fields for each file include: file_name, variable, src_short, summary_type, label 
    * File name must match corresponsing var file name, shapefiles can be represented by only the .shp file
    * See more information and download directions here:

In [1]:
#List of URLS with files for Land Cover data
#Note was having trouble downloading these using Python, server kept cutting off access
import urllib.request as url_r

try:
    target_url = 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/manifest_cgls_lc_v2_100m_global_2015.txt'
    download_urls = url_r.urlopen(target_url).read()
    download_urls_str = download_urls.decode("utf8")
    list_urls = list(download_urls_str.split('\r\n'))
except Exception as e:
    raise Exception(e)

In [2]:
#Shows first 4 urls
list_urls[0:4]

['https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N00_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip',
 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N20_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip',
 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N40_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip',
 'https://s3-eu-west-1.amazonaws.com/vito-lcv/2015/ZIPfiles/E000N60_ProbaV_LC100_epoch2015_global_v2.0.1_products_EPSG-4326.zip']

==================================================================================================================
******************************************************************************************************************
==================================================================================================================

## Step 2: Create intermediate files containing data needed for processing steps

### A) create file: f'{directory}/{short_name}_file_info.json'  -> example   data/var/tif_var_file_info.json
    * Example below creates two json files  'data/var/tif_vars_file_info.json' and 'data/var/shp_vars_file_info.json'.  The information in this file is used in later steps to inform how tif and shp data from our source landscape variable datasets (stored in data/var directory in the repository) are attributed to basins in HydroSHEDS HYDROBASINS

In [3]:
from utils import file_management as f_mng

directory = 'data/var'
file_types = ['.shp','.tif']


for file_type in file_types:
    short_name = f'{file_type[1:]}_vars'
    file_list, directory = f_mng.find_files(directory, suffix=file_type)

    #convert file information to dictionary and export to json
    file_info, missing_info = f_mng.store_file_info(file_list, directory, short_name)

______________________________________________________________________________________________
#### Below is a count of the number of .tif files ready to be processed and a view of the information for the first 5 files.

Note: File processing information is available in two formats.  The function passes back a list of dictionaries.  Each dictionary stores information about a file.  The same information is also saved to disk as 'data/var/tif_vars_file_info.json', for processing in later sessions.  Here is an example of printing the number of files processed and showing the first 5 records using the returned variable file_info.  

In [4]:
print (f'{len(file_info)} files are documented and ready to process \n')

import pandas as pd
pd.DataFrame(file_info[0:5])

941 files are documented and ready to process 



,file_name,file_path,label,src_short,summary_type,variable
0,gpw_v4_population_density_rev11_2015_30_sec.tif,data/var/Gridded Population of the World_v4.11...,gpw2015_pop_den,gpw2015_v4_rev11_30_sec,count mean nodata,population_density
1,W120N80_ProbaV_LC100_epoch2015_global_v2.0.1_w...,data/var/LC100_epoch2015/W120N80_ProbaV_LC100_...,lc2015_waterseas_cov,lc100_epoch2015_v2.0.1,count mean nodata,water-seasonal-coverfraction-layer
2,W120N80_ProbaV_LC100_epoch2015_global_v2.0.1_m...,data/var/LC100_epoch2015/W120N80_ProbaV_LC100_...,lc2015_moss_cov,lc100_epoch2015_v2.0.1,count mean nodata,moss-coverfraction-layer
3,W120N80_ProbaV_LC100_epoch2015_global_v2.0.1_c...,data/var/LC100_epoch2015/W120N80_ProbaV_LC100_...,lc2015_crops_cov,lc100_epoch2015_v2.0.1,count mean nodata,crops-coverfraction-layer
4,W120N80_ProbaV_LC100_epoch2015_global_v2.0.1_t...,data/var/LC100_epoch2015/W120N80_ProbaV_LC100_...,lc2015_tree_cov,lc100_epoch2015_v2.0.1,count mean nodata,tree-coverfraction-layer


_____________________________________________________________________________________________________________
#### The process also returned the variable missing_info, which is a list of dictionaries holding information about .tif files that were available and meeting the criteria (i.e. specified directory and file_type) yet were either not in the file_processing_info.csv file or were in the file but had missing values.

In [5]:
#Example of how to use missing_info to understand what files that met criteria are not currently ready for processing

#Import file_processing_info to get a count of number of columns
import pandas as pd
df = pd.read_csv('data/var/file_processing_info.csv')
rows, cols = df.shape

#If missing_info has data, count and print number of files that are not in the csv.
#Also count and print number of files that are in the csv but have missing data.  In this case print the file name and missing fields.
if missing_info:
    w_missing_file = 0
    w_missing_fields = 0
    for record in missing_info:
        if 'missing_fields' in record and len(record['missing_fields'])==cols:
            w_missing_file += 1
        elif 'missing_fields' in record and len(record['missing_fields'])<cols:
            w_missing_fields += 1
            print (f"{record['file_name']} found in file_processing_info.csv but has missing fields: {record['missing_fields']}")
            print ('\n')
        elif 'file_failed' in record:
            print (f"{record['file_name']} failed")
            print ('\n')
    print (f'{w_missing_fields} file(s) with missing fields in data/var/file_processing_info.csv')
    print (f'{w_missing_file} files(s) are not in data/var/file_processing_info.csv')
else:
    print('No files with missing processing information detected in processed directory')

0 file(s) with missing fields in data/var/file_processing_info.csv
3178 files(s) are not in data/var/file_processing_info.csv


### B) Create serlialized versions of HydroSHEDS level 12 basins 

3 variations are created and pickled for future use allowing for quick access to consistent information throughout processing steps
    * geodataframe containing all attributes and all geospatial information (poly)
    * dataframe containing all attributes except geospatial information
    * list of hybas_ids 

In [6]:
from utils import file_management as f_mng
f_mng.build_basin_data(level='12', version='v1c', directory = 'data/HydroSHEDS')
#Prints number of records

1034083


In [7]:
#Test read gdf
pkl_gdf = 'data/basins_lvl12_gdf.pkl'
gdf = f_mng.read_pkl_gdf(pkl_gdf)
gdf.head(5)

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT,geometry
0,1120000010,0,1120000010,1120000010,0.0,0.0,11.0,11.0,111011001000,0,1,0,1,"POLYGON ((32.50000000000002 29.94583333333337,..."
1,1120000020,0,1120000020,1120000020,0.0,0.0,137.0,416.8,111011002100,0,0,1,2,"POLYGON ((32.36250000000002 29.97083333333337,..."
2,1121694330,1120000020,1120000020,1120000020,19.5,19.5,135.1,280.0,111011002200,0,0,1,3,"POLYGON ((32.36250000000002 29.9666666666667, ..."
3,1121693980,1121694330,1120000020,1120000020,35.3,35.3,144.9,144.9,111011002300,0,0,1,4,"POLYGON ((32.25833333333335 29.9916666666667, ..."
4,1120000030,0,1120000030,1120000030,0.0,0.0,186.8,186.9,111011003000,0,1,0,5,"POLYGON ((32.40000000000002 29.73750000000003,..."


In [8]:
gdf.shape

(1034083, 14)

In [9]:
#Test read df
pkl_df_path = 'data/basins_lvl12_df.pkl'
df=f_mng.read_pkl_df(pkl_df_path)
df.head()

,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,PFAF_ID,ENDO,COAST,ORDER,SORT
0,1120000010,0,1120000010,1120000010,0.0,0.0,11.0,11.0,111011001000,0,1,0,1
1,1120000020,0,1120000020,1120000020,0.0,0.0,137.0,416.8,111011002100,0,0,1,2
2,1121694330,1120000020,1120000020,1120000020,19.5,19.5,135.1,280.0,111011002200,0,0,1,3
3,1121693980,1121694330,1120000020,1120000020,35.3,35.3,144.9,144.9,111011002300,0,0,1,4
4,1120000030,0,1120000030,1120000030,0.0,0.0,186.8,186.9,111011003000,0,1,0,5


In [10]:
df.shape

(1034083, 13)

In [12]:
#Test read list
pkl_df_path = 'data/basins_lvl12.txt'
hybas_id_list=f_mng.read_pkl_df(pkl_df_path)
hybas_id_list[0:5]

[1121976320, 4120903680, 3120562180, 8120172550, 2120220680]